In [ ]:
%%writefile mat.cu
#include <iostream>
using namespace std;

// CUDA code to multiply matrices
__global__ void multiply(int* A, int* B, int* C, int size) {
    // Uses thread indices and block indices to compute each element
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;

        // Print thread index and operation
        printf("Thread (%d, %d) performed multiplication for C[%d][%d]\n", threadIdx.x, threadIdx.y, row, col);
    }
}

// Initialize matrix C with zeros
void initializeZero(int* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = 0;
    }
}

void print(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            cout << matrix[row * size + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}

int main() {
    int N;
    cout << "Enter the size of the matrices: ";
    cin >> N;

    int* A, * B, * C;

    int matrixSize = N * N;
    size_t matrixBytes = matrixSize * sizeof(int);

    A = new int[matrixSize];
    B = new int[matrixSize];
    C = new int[matrixSize];

    cout << "Enter the elements of matrix A: \n";
    for (int i = 0; i < matrixSize; ++i) {
        cin >> A[i];
    }

    cout << "Enter the elements of matrix B: \n";
    for (int i = 0; i < matrixSize; ++i) {
        cin >> B[i];
    }

    cout << "Matrix A: \n";
    print(A, N);

    cout << "Matrix B: \n";
    print(B, N);


    int* X, * Y, * Z;
    // Allocate space
    cudaMalloc(&X, matrixBytes);
    cudaMalloc(&Y, matrixBytes);
    cudaMalloc(&Z, matrixBytes);

    // Copy values from A to X
    cudaMemcpy(X, A, matrixBytes, cudaMemcpyHostToDevice);

    // Copy values from A to X and B to Y
    cudaMemcpy(Y, B, matrixBytes, cudaMemcpyHostToDevice);

    // Threads per CTA dimension
    int THREADS = 16;

    // Blocks per grid dimension
    int BLOCKS = (N + THREADS - 1) / THREADS;

    // Use dim3 structs for block and grid dimensions
    dim3 threads(THREADS, THREADS);
    dim3 blocks(BLOCKS, BLOCKS);

    // Launch kernel
    multiply<<<blocks, threads>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, matrixBytes, cudaMemcpyDeviceToHost);
    cout << "Multiplication of matrix A and B: \n";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}


Writing mat.cu


In [ ]:

!nvcc mat.cu -o mat

In [ ]:
!./mat

Enter the size of the matrices: 3
Enter the elements of matrix A: 
1 2 3 4 5 6 7 8 9
Enter the elements of matrix B: 
1 2 1 2 4 6 7 2 5
Matrix A: 
1 2 3 
4 5 6 
7 8 9 

Matrix B: 
1 2 1 
2 4 6 
7 2 5 

Multiplication of matrix A and B: 
0 0 0 
0 0 0 
0 0 0 



In [ ]:
%%writefile add.cu

#include <iostream>
#include <cstdlib> // Include <cstdlib> for rand()
using namespace std;

__global__ void add(int* A, int* B, int* C, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        C[tid] = A[tid] + B[tid];
        printf("Thread %d: %d + %d = %d\n", tid, A[tid], B[tid], C[tid]);
    }
}

void print(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        cout << vector[i] << " ";
    }
    cout << endl;
}

int main() {
    int N;
    cout << "Enter the size of the vectors: ";
    cin >> N;

    // Allocate host memory
    int* A = new int[N];
    int* B = new int[N];
    int* C = new int[N];

    // Initialize host arrays
    cout << "Enter elements for vector A: ";
    for (int i = 0; i < N; i++) {
        cin >> A[i];
    }

    cout << "Enter elements for vector B: ";
    for (int i = 0; i < N; i++) {
        cin >> B[i];
    }

    cout << "Vector A: ";
    print(A, N);
    cout << "Vector B: ";
    print(B, N);

    int* X, * Y, * Z;
    size_t vectorBytes = N * sizeof(int);

    // Allocate device memory
    cudaMalloc(&X, vectorBytes);
    cudaMalloc(&Y, vectorBytes);
    cudaMalloc(&Z, vectorBytes);

    // Copy data from host to device
    cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Launch kernel
    add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);

    // Copy result from device to host
    cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);

    cout << "Addition: ";
    print(C, N);

    // Free device memory
    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    // Free host memory
    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


Overwriting add.cu


In [ ]:
!nvcc add.cu -o add

In [ ]:
!./add

Enter the size of the vectors: 5
Enter elements for vector A: 3 4 6 7 8 9
Enter elements for vector B: 9 8
7
6
Vector A: 3 4 6 7 8 
Vector B: 9 9 8 7 6 
Thread 0: 3 + 9 = 12
Thread 1: 4 + 9 = 13
Thread 2: 6 + 8 = 14
Thread 3: 7 + 7 = 14
Thread 4: 8 + 6 = 14
Addition: 12 13 14 14 14 
